In [1]:
from __future__ import print_function

import numpy as np
import time
from keras.preprocessing.image import save_img
from keras.applications import vgg16
from keras import backend as K
from keras.models import load_model

import tensorflow as tf

Using TensorFlow backend.


In [2]:
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize

In [3]:
# Define IoU metric
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

In [11]:
# dimensions of the generated pictures for each filter.
img_width = 128
img_height = 128

# the name of the layer we want to visualize
# (see model definition at keras/applications/vgg16.py)
layer_name = 'c4'

# util function to convert a tensor into a valid image


def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + K.epsilon())
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_data_format() == 'channels_first':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x


model = load_model('../../modele/unet_on_coco.h5', custom_objects={'mean_iou': mean_iou})
print('Model loaded.')

#model.summary()

# this is the placeholder for the input images
input_img = model.input

# get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])


def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + K.epsilon())

list_of_registere_arr = np.zeros((1, 128, 128, 3), dtype=np.uint8)
    
registered_arr = np.zeros((480, 360, 3), dtype=np.uint8)

kept_filters = []
for filter_index in range(64):
    # we only scan through the first 12 filters,
    # but there are actually 16 of them
    print('Processing filter %d' % filter_index)
    start_time = time.time()

    # we build a loss function that maximizes the activation
    # of the nth filter of the layer considered
    layer_output = layer_dict[layer_name].output
    if K.image_data_format() == 'channels_first':
        loss = K.mean(layer_output[:, filter_index, :, :])
    else:
        loss = K.mean(layer_output[:, :, :, filter_index])
    
    # we compute the gradient of the input picture wrt this loss
    grads = K.gradients(loss, input_img)[0]

    # normalization trick: we normalize the gradient
    grads = normalize(grads)

    # this function returns the loss and grads given the input picture
    iterate = K.function([input_img], [loss, grads])

    # step size for gradient ascent
    step = 1.

    # we start from a gray image with some random noise
    if K.image_data_format() == 'channels_first':
        input_img_data = np.random.random((1, 3, img_width, img_height))
    else:
        input_img_data = np.random.random((1, img_width, img_height, 3))
        
   
    
    registered_arr = imread('ziem.jpg')
    
    registered_lower_res = np.zeros((128, 128, 3), dtype=np.uint8)
    
    registered_lower_res = resize(registered_arr, (128,128), mode='constant', preserve_range=True)
    list_of_registere_arr[0] = registered_lower_res
    
    input_img_data = (list_of_registere_arr - 0.5) * 20 + 128
    
    # we run gradient ascent for 20 steps
    for i in range(20):
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value * step

        #print('Current loss value:', loss_value)
        if loss_value <= 0.:
            # some filters get stuck to 0, we can skip them
            break

    # decode the resulting input image
    if loss_value > 0:
        img = deprocess_image(input_img_data[0])
        kept_filters.append((img, loss_value))
    end_time = time.time()
    print('Filter %d processed in %ds' % (filter_index, end_time - start_time))



Model loaded.
Processing filter 0
Filter 0 processed in 2s
Processing filter 1
Filter 1 processed in 2s
Processing filter 2
Filter 2 processed in 2s
Processing filter 3
Filter 3 processed in 2s
Processing filter 4
Filter 4 processed in 2s
Processing filter 5
Filter 5 processed in 2s
Processing filter 6
Filter 6 processed in 2s
Processing filter 7
Filter 7 processed in 2s
Processing filter 8
Filter 8 processed in 2s
Processing filter 9
Filter 9 processed in 2s
Processing filter 10
Filter 10 processed in 2s
Processing filter 11
Filter 11 processed in 2s
Processing filter 12
Filter 12 processed in 2s
Processing filter 13
Filter 13 processed in 3s
Processing filter 14
Filter 14 processed in 3s
Processing filter 15
Filter 15 processed in 3s
Processing filter 16
Filter 16 processed in 3s
Processing filter 17
Filter 17 processed in 3s
Processing filter 18
Filter 18 processed in 3s
Processing filter 19
Filter 19 processed in 3s
Processing filter 20
Filter 20 processed in 3s
Processing filter 2

In [13]:
# we will stich the best 64 filters on a 8 x 8 grid.
n = 5
print(len(kept_filters) )
# the filters that have the highest loss are assumed to be better-looking.
# we will only keep the top 64 filters.
kept_filters.sort(key=lambda x: x[1], reverse=True)
kept_filters = kept_filters[:n * n]

# build a black picture with enough space for
# our 8 x 8 filters of size 128 x 128, with a 5px margin in between
margin = 5
width = n * img_width + (n - 1) * margin
height = n * img_height + (n - 1) * margin
stitched_filters = np.zeros((width, height, 3))

# fill the picture with our saved filters
for i in range(n):
    for j in range(n):
        img, loss = kept_filters[i * n + j]
        width_margin = (img_width + margin) * i
        height_margin = (img_height + margin) * j
        stitched_filters[
            width_margin: width_margin + img_width,
            height_margin: height_margin + img_height, :] = img

# save the result to disk
save_img('filters/' + layer_name + 'stitched_filters_%dx%d.png' % (n, n), stitched_filters)

54
